In [1]:
import polars as pl
from pprint import pprint

In [2]:
def fix_str_to_int(data:pl.DataFrame, fix_list:list, type:any=pl.Int64):
    data = data.with_columns(pl.col(fix_list).str.split(","))
    data = data.with_columns(pl.col(fix_list).list.eval(pl.element().str.strip()))
    data = data.with_columns(pl.col(fix_list).list.eval(pl.element().cast(pl.Int64, strict=False)))

    for column in fix_list:
        data = data.with_columns(pl.when(pl.col(column).list.eval(pl.element().is_null()).list.any()).then(None).otherwise(pl.col(column)).alias("bools"))
        data = data.select(replace(data, data.get_column_index(column), "bools")).rename({"bools":column})
    
    # data = data.with_columns(pl.col(fix_list).str.strip().str.replace("","0")).cast(conversion_dict)

    return data


# Abych nebyl nařčen z plagiátorství, zdroj: https://stackoverflow.com/questions/75954280/how-to-change-the-position-of-a-single-column-in-python-polars-library
def replace(df, new_position, col_name):
    neworder=df.columns
    neworder.pop(new_position)
    neworder.remove(col_name)
    neworder.insert(new_position,col_name)
    return neworder


In [3]:
rozvrh_by_kat = pl.read_csv("source_tables/rozvrh_katedry.csv").drop("semestr").unique().rename({"predmet" : "zkratka"})
rozvrh_by_kat = fix_str_to_int(rozvrh_by_kat, ["ucitIdno.ucitel", "vsichniUciteleUcitIdno"])
#print(rozvrh_by_kat.head())
predmety_by_kat = pl.read_csv("source_tables/predmety_katedry.csv").drop("semestr").unique().drop(["pocetStudentu", "aSkut", "bSkut", "cSkut"])
predmety_by_kat = fix_str_to_int(predmety_by_kat, ["garantiUcitIdno", "prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "hodZaSemKombForma"])
predmety_by_kat.head(10)
#predmety_by_kat = pl.read_excel("source_tables/getPredmetyByKatedraFullInfo-2023-03-15-17-40.xlsx", sheet_id=1, engine="openpyxl").drop("semestr").unique().drop(["pocetStudentu", "aSkut", "bSkut", "cSkut"])

C:\Users\Matej\AppData\Local\Temp\ipykernel_15868\651153448.py:3: DeprecationWarning: `strip` is deprecated. It has been renamed to `strip_chars`.
  data = data.with_columns(pl.col(fix_list).list.eval(pl.element().str.strip()))


katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""AFJ""",2023,"""Teorie automat…","""Teorie automat…","""A""","""N""",5,"""NE""","""""","""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…","[449, 2220]","""""","""""",null,"""""","""""","""""","""""","""""","""KI/AFJ1, KI/TA…","""""","""'Chytil M. Aut…","""""","""""","""""","""N""",2,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""V tomto předmě…","""Zkouška""","""ANO""","""Kombinovaná""","""zápočet písemn…","""1. Konečné aut…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""","""0"""
"""KI""","""AIT""",2023,"""Architektura a…","""Architektura a…","""A""","""N""",3,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""""","""""",null,"""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""","""""","""""","""KI/ZPP, KI/ZPS…","""""","""KI/IVP""","""'Velte, A. T.,…","""""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",2,"""HOD/TYD""","""Kurz je zaměře…","""Zkouška""","""NE""","""Kombinovaná""","""Zápočet: labor…","""1. IT a její ž…","""Základny počít…","""""","""""","""""","""Čeština""","""Tento kurz vzn…","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""ANE""",2023,"""Analogová elek…","""Analogová elek…","""A""","""N""",2,"""NE""","""""","""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""""","""""",null,"""""","""""","""""","""""","""KI/ZEL""","""""","""""","""'Tichý, M. Ele…","""""","""""","""""","""N""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Kurz je zaměře…","""Zápočet""","""NE""","""Kombinovaná""","""Ústní obhajoba…","""1. Pasivní ele…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""APC""",2023,"""Architektura p…","""Architektura p…","""A""","""N""",3,"""NE""","""""","""'doc. RNDr. Mg…","""'doc. RNDr. Mg…",[2239],"""'Ing. Pavel Ku…","""'Ing. Pavel Ku…","[2317, 2239]","""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""KI/DHW, KI/PPT…","""'Horák J. Hard…","""""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Tento předmět …","""Zkouška""","""NE""","""Kombinovaná""","""Zkouška je kom…","""1) Vývoj PC, g…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""",""

In [4]:
# qr = ~pl.all().is_duplicated()
# rozvrh_by_kat = rozvrh_by_kat.filter(qr)
# predmety_by_kat = predmety_by_kat.filter(qr)


In [5]:
predmety_by_kat.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""AFJ""",2023,"""Teorie automat…","""Teorie automat…","""A""","""N""",5,"""NE""","""""","""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…","[449, 2220]","""""","""""",null,"""""","""""","""""","""""","""""","""KI/AFJ1, KI/TA…","""""","""'Chytil M. Aut…","""""","""""","""""","""N""",2,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""V tomto předmě…","""Zkouška""","""ANO""","""Kombinovaná""","""zápočet písemn…","""1. Konečné aut…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""","""0"""
"""KI""","""AIT""",2023,"""Architektura a…","""Architektura a…","""A""","""N""",3,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""""","""""",null,"""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""","""""","""""","""KI/ZPP, KI/ZPS…","""""","""KI/IVP""","""'Velte, A. T.,…","""""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",2,"""HOD/TYD""","""Kurz je zaměře…","""Zkouška""","""NE""","""Kombinovaná""","""Zápočet: labor…","""1. IT a její ž…","""Základny počít…","""""","""""","""""","""Čeština""","""Tento kurz vzn…","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""ANE""",2023,"""Analogová elek…","""Analogová elek…","""A""","""N""",2,"""NE""","""""","""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""""","""""",null,"""""","""""","""""","""""","""KI/ZEL""","""""","""""","""'Tichý, M. Ele…","""""","""""","""""","""N""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Kurz je zaměře…","""Zápočet""","""NE""","""Kombinovaná""","""Ústní obhajoba…","""1. Pasivní ele…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""APC""",2023,"""Architektura p…","""Architektura p…","""A""","""N""",3,"""NE""","""""","""'doc. RNDr. Mg…","""'doc. RNDr. Mg…",[2239],"""'Ing. Pavel Ku…","""'Ing. Pavel Ku…","[2317, 2239]","""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""KI/DHW, KI/PPT…","""'Horák J. Hard…","""""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Tento předmět …","""Zkouška""","""NE""","""Kombinovaná""","""Zkouška je kom…","""1) Vývoj PC, g…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""",""

In [6]:
rozvrh_by_kat.head(10)
#rozvrh_by_kat.__len__()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
i64,str,str,str,str,str,list[i64],str,str,str,str,str,str,str,i64,str,f64,i64,i64,i64,str,str,str,str,str,str,i64,i64,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,list[i64],str,str,str,i64,str,str,str,str
486683,"""Operační systé…","""KI""","""OPS""","""""","""650""",[650],"""Petr""","""Kubera""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""MF""",2.28,190,140,140,"""Přednáška""","""Př""","""A""","""Čtvrtek""","""Čt""","""""",8,8,1,"""14:00""","""14:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.API Bc,1.MvF…","""UJEP""","""11.4.2024""","""""","""""","""R""",[650],"""RNDr. Petr Kub…","""'RNDr. Petr Ku…","""Kubera""",486683,"""""","""""","""RSKVARA""",""""""
487024,"""Datová a infor…","""KI""","""DIB""","""""","""2776""",[2776],"""Jan""","""Krejčí""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""",7.07,12,8,7,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""",2,2,1,"""08:00""","""08:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.API N""","""UJEP""","""12.4.2024""","""""","""""","""R""",[2776],"""RNDr. Jan Krej…","""'RNDr. Jan Kre…","""Krejčí""",487024,"""""","""""","""RSKVARA""",""""""
479612,"""Programování -…","""KI""","""KPYT""","""""","""6259""",[6259],"""Jiří""","""Škvára""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""",6.13,30,5,2,"""Cvičení""","""Cv""","""A""","""Sobota""","""So""","""""",7,9,3,"""13:00""","""15:50""",15,15,"""Jiný""","""J""","""31867""","""N""","""N""","""Automaticky vy…","""2.PMVTBcKS""","""UJEP""","""13.4.2024""","""""","""""","""R""",[6259],"""RNDr. Jiří Škv…","""'RNDr. Jiří Šk…","""Škvára""",479612,"""""","""""","""RSKVARA""",""""""
491279,"""Datová a infor…","""KI""","""KDIB""","""""","""2776""",[2776],"""Jan""","""Krejčí""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""",7.07,12,7,4,"""Cvičení""","""Cv""","""A""","""Sobota""","""So""","""""",10,11,2,"""16:00""","""17:50""",15,15,"""Jiný""","""J""","""31988""","""N""","""N""","""Automaticky vy…","""1.KAPI N""","""UJEP""","""13.4.2024""","""""","""""","""R""",[2776],"""RNDr. Jan Krej…","""'RNDr. Jan Kre…","""Krejčí""",491279,"""""","""""","""RSKVARA""",""""""
487005,"""Programování p…","""KI""","""PRI""","""""","""2239""",[2239],"""Viktor""","""Maškov""","""doc. RNDr. Mgr…","""DrSc.""","""A""","""A""","""100""",2023,"""CP""",6.14,31,44,42,"""Přednáška""","""Př""","""A""","""Středa""","""St""","""""",2,3,2,"""08:00""","""09:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""2.API Bc,3.API…","""UJEP""","""17.4.2024""","""""","""""","""R""",[2239],"""doc. RNDr. Mgr…","""'doc. RNDr. Mg…","""Maškov""",487005,"""""","""""","""RSKVARA""",""""""
487003,"""Programování p…","""KI""","""GUI""","""""","""650""",[650],"""Petr""","""Kubera""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""",6.13,30,35,30,"""Přednáška""","""Př""","""A""","""Čtvrtek""","""Čt""","""""",2,3,2,"""08:00""","""09:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""2.API Bc,3.API…","""UJEP""","""18.4.2024""","""""","""""","""R""","[650, 6973]","""RNDr. Petr Kub…","""'RNDr. Petr Ku…","""Kubera, Beráne…",487003,"""""","""""","""RSKVARA""",""""""
486683,"""Operační systé…","""KI""","""OPS""","""""","""650""",[650],"""Petr""","""Kubera""","""RNDr.

In [7]:
set(rozvrh_by_kat.columns) & set(predmety_by_kat.columns)

{'katedra', 'nazev', 'rok', 'zkratka'}

In [8]:
rozvrh_by_kat.null_count()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
2,0,0,0,0,0,41,0,0,0,0,0,0,0,0,0,146,146,2,0,0,0,0,0,0,0,151,151,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41,0,0,0,0,0,0,0,0


In [9]:
predmety_by_kat.null_count()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,45,0,0,308,0,0,291,0,0,295,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,231,0,0,0


In [10]:
# vybírám pouze předměty, které mají rozvrhovou akci

In [11]:
predmety_s_akci = predmety_by_kat.join(other=rozvrh_by_kat, on="zkratka", how="inner").select(predmety_by_kat.columns).unique().sort("zkratka")
str(len(predmety_s_akci)) + "   " + str(len(predmety_by_kat)) # ...what does this do
predmety_s_akci.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""0033""",2023,"""Optimalizace v…","""Optimalizace v…","""A""","""A""",2,"""NE""","""""","""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""""","""""",null,"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Lagová M., Ja…","""""","""""","""""","""N""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""Náplní výběrov…","""Zápočet""","""NE""","""Ústní""","""Vstupní požada…","""1) Úvod do mat…","""""","""""","""""","""""","""Čeština""","""vk 1528""","""A""","""N""","""N""","""""","""1""","""1""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""0121""",2023,"""CCNA 1: Introd…","""CCNA 1: Introd…","""A""","""A""",1,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem předmětu…","""Zápočet""","""NE""","""Kombinovaná""","""Splnění závěre…","""Cílem předmětu…","""""","""""","""""","""""","""Čeština, Angli…","""vk""","""A""","""N""","""N""","""""","""""","""""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""0126""",2023,"""CCNA 2: Routin…","""CCNA 2: Routin…","""A""","""A""",2,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem předmětu…","""Zápočet""","""NE""","""Kombinovaná""","""Splnění závěre…","""Cílem předmětu…","""CCNA 1""","""""","""""","""""","""Čeština, Angli…","""vk""","""A""","""N""","""N""","""""","""""","""""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""0127""",2023,"""CCNA 3: Scalin…","""CCNA 3: Scalin…","""A""","""A""",2,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem předmětu…","""Zápočet""","""NE""","""Kombinovaná""","""Splnění závěre…","""Cíle předmětu:…","""CCNA 2""","""""","""""","""""","""Čeština, Angli…","""vk""","""A""","""N""","""N""","""""","""""","""""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""0190""",2023,"""CCNA Security""","""CCNA Security""","""A""","""A""",2,"""NE""","""""","""'

# Vyučující a garanti

In [12]:
jednotek_prednasek = pl.col("jednotekPrednasek")
jednotek_cviceni = pl.col("jednotekCviceni")
jednotek_seminare = pl.col("jednotekSeminare")
garant = pl.col("garanti")
cvicici = pl.col("cvicici")
prednasejici = pl.col("prednasejici")
seminarici = pl.col("seminarici")

### Každý předmět má právě jednoho garanta

> Zde jsou předměty, které nemají garanta:

In [13]:
zkratky = predmety_s_akci.filter(pl.col("garantiUcitIdno").is_null()).select(["zkratka", "rok", "nazev", "nazevDlouhy", "garanti", "garantiSPodily"]).filter(
    # Není SZ
    # pl.col("zkratka").str.starts_with("SZ").is_not()
    True
)
print(f"\n{len(zkratky)} Předmětů nemá zadaného garanta")
zkratky.write_excel("results/bez_garanta.xlsx")
zkratky
# print(list(zkratky), sep=", ")


17 Předmětů nemá zadaného garanta


zkratka,rok,nazev,nazevDlouhy,garanti,garantiSPodily
str,i64,str,str,str,str
"""BOP""",2023,"""Odborná praxe""","""Odborná praxe""","""""",""""""
"""BP2O""",2023,"""Bakalářská prá…","""Bakalářská prá…","""""",""""""
"""BPP""",2023,"""Bakalářská prá…","""Bakalářská prá…","""""",""""""
"""DPP""",2023,"""Diplomová prác…","""Diplomová prác…","""""",""""""
"""EDAV""",2023,"""Data Analysis …","""Data Analysis …","""""",""""""
…,…,…,…,…,…
"""SZZIV""",2023,"""Informatika pr…","""Informatika pr…","""""",""""""
"""SZZPP""",2023,"""Aplikovaná inf…","""Aplikovaná inf…","""""",""""""
"""SZZTD""",2023,"""Teoretické zák…","""Teoretické zák…","""""",""""""


> Předměty podezřelé z více garantů:

In [14]:
# predmety_s_akci.filter( > pl.lit())
# Nefunguje, arr.lengths údajně nic není
vice_garantu = predmety_s_akci.with_columns(pl.col("garantiUcitIdno").list.len().alias("pocet garantu")
    ).select(
        ["garantiUcitIdno", "garanti", "zkratka", "pocet garantu"]
    ).filter(pl.col("pocet garantu") > 1)
vice_garantu.write_excel("results/vice_garantu.xlsx")
vice_garantu.head(10)

garantiUcitIdno,garanti,zkratka,pocet garantu
list[i64],str,str,u32


### Každý předmět má alespoň jednoho vyučujícího v kategoriích dle parametrů předmětu

např. pokud má předmět rozsah 2 h přednášek a 2 h cvičení, pak musí mít alespoň jednoho přednášejícího a alespoň jednoho cvičícího a zároveň nesmí mít vyučujícího v kategorii vede seminář.

In [15]:
ma_prednasku = jednotek_prednasek != 0
nema_prednasejiciho = pl.col("prednasejiciUcitIdno").is_null()

chybi_prednasejici = predmety_s_akci.filter(ma_prednasku & nema_prednasejiciho)
chybi_prednasejici.write_excel("results/chybi_prednasejici.xlsx")
chybi_prednasejici.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str


In [16]:
ma_cviceni = jednotek_cviceni != 0
nema_cviciciho = pl.col("cviciciUcitIdno").is_null()

chybi_cvicici = predmety_s_akci.filter(ma_cviceni & nema_cviciciho)
chybi_cvicici.write_excel("results/chybi_cvicici.xlsx")
chybi_cvicici.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""BOP""",2023,"""Odborná praxe""","""Odborná praxe""","""A""","""N""",24,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""A""",0,"""HOD/TYD""",12,"""TYD/SEM""",0,"""HOD/TYD""","""Odborná praxe …","""Zápočet""","""NE""","""Kombinovaná""","""Zápočet je udě…","""Odborná praxe …","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""


In [17]:
ma_seminare = jednotek_seminare != 0
nema_seminariciho = pl.col("seminariciUcitIdno").is_null()

chybi_seminarici = predmety_s_akci.filter(ma_seminare & nema_seminariciho)
chybi_seminarici.write_excel("results/chybi_seminarici.xlsx")
chybi_seminarici

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""KAFJ""",2023,"""Teorie automat…","""Teorie automat…","""A""","""N""",5,"""NE""","""""","""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…","[449, 2220]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Chytil M. Aut…","""""","""""","""""","""N""",6,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""V tomto předmě…","""Zkouška""","""ANO""","""Kombinovaná""","""Požadavky pro …","""1. Konečné aut…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""","""0"""
"""KI""","""KAPR1""",2023,"""Algoritmizace …","""Algoritmizace …","""A""","""N""",4,"""NE""","""""","""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""'Ing. Mgr. Pav…","""'Ing. Mgr. Pav…","[6973, 650, … 6259]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""KI/KAPR2, KI/K…","""'https://githu…","""""","""""","""""","""N""",6,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""Úvodní kurz al…","""Zápočet""","""NE""","""Kombinovaná""","""Podmínkou získ…","""1. Základní te…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""KAPR2""",2023,"""Algoritmizace …","""Algoritmizace …","""N""","""A""",5,"""NE""","""""","""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""'RNDr. Petr Ku…","""'RNDr. Petr Ku…","[650, 835, 6259]","""""","""""",null,"""""","""""","""""","""""","""KI/KAPR1""","""""","""KI/KGUI, KI/KP…","""'https://githu…","""""","""""","""""","""N""",6,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""Druhá část úvo…","""Zkouška""","""ANO""","""Kombinovaná""","""Podmínkou získ…","""1. Implementac…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""KCSY""",2023,"""Číslicové syst…","""Číslicové syst…","""N""","""A""",4,"""NE""","""""","""'prof. Sergii …","""'prof. Sergii …",[3416],"""'prof. Sergii …","""'prof. Sergii …",[3416],"""'prof. Sergii …","""'prof. Sergii …","[3416, 1780]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""KI/PCSY""","""'Gook M. Hardw…","""""","""""","""""","""N""",4,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""Student se v r…","""Zkouška""","""ANO""","""Kombinovaná""","""Zápočet: semes…","""1. Syntéza kom…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""

> Zde jsou předměty, ve kterých učitel vede seminář:

In [18]:
#prednasejici_je_seminarici = prednasejici == seminarici
#cvicici_je_seminarici = cvicici == seminarici

#konflikt_vyucujicich = predmety_s_akci.filter(ma_seminare & ma_prednasku & (prednasejici_je_seminarici | cvicici_je_seminarici))
#konflikt_vyucujicich.write_excel("results/konflikt_vyucujicich.xlsx")
#konflikt_vyucujicich.head()

In [19]:
print(f"{len(chybi_prednasejici)} předmětů nemá přednášejícího")
print(f"{len(chybi_cvicici)} předmětů nemá cvičícího")
print(f"{len(chybi_seminarici)} předmětů nemá seminářícího")
# print(f"U {len(konflikt_vyucujicich)} předmětů je seminářící také učitel")

0 předmětů nemá přednášejícího
1 předmětů nemá cvičícího
29 předmětů nemá seminářícího


### Garant by měl být někde mezi vyučujícími. Pokud má předmět přednášku, měl by být přednášející.

> V těchto předmětech garant vůbec neučí:

In [20]:
filter_query = (garant != cvicici) & (garant != seminarici) & (garant != prednasejici) & (ma_prednasku | ma_cviceni | ma_seminare)
predmety_kde_garant_neuci = predmety_s_akci.filter(filter_query)
predmety_kde_garant_neuci.write_excel("results/predmety_kde_garant_neuci.xlsx")
predmety_kde_garant_neuci.__len__()

55

> v těchto předmětech je přednáška, ale nedělá jí garant:

In [21]:
garant_neni_prednasejici = (jednotek_prednasek != 0) & (garant != prednasejici)
garant_neprednasi = predmety_s_akci.filter(garant_neni_prednasejici)
garant_neprednasi.write_excel("results/garant_neprednasi.xlsx")
garant_neprednasi.__len__()

27

## Kontrola proti rozvrhovaným akcím

### Vyučující u rozvrhových akcí musí být v příslušných kategoriích u vyučujících

#### Osekám dataframy

In [22]:
rozvrh_by_kat.head()
# predmety_by_kat.head()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
i64,str,str,str,str,str,list[i64],str,str,str,str,str,str,str,i64,str,f64,i64,i64,i64,str,str,str,str,str,str,i64,i64,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,list[i64],str,str,str,i64,str,str,str,str
486683,"""Operační systé…","""KI""","""OPS""","""""","""650""",[650],"""Petr""","""Kubera""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""MF""",2.28,190,140,140,"""Přednáška""","""Př""","""A""","""Čtvrtek""","""Čt""","""""",8,8,1,"""14:00""","""14:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.API Bc,1.MvF…","""UJEP""","""11.4.2024""","""""","""""","""R""",[650],"""RNDr. Petr Kub…","""'RNDr. Petr Ku…","""Kubera""",486683,"""""","""""","""RSKVARA""",""""""
487024,"""Datová a infor…","""KI""","""DIB""","""""","""2776""",[2776],"""Jan""","""Krejčí""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""",7.07,12,8,7,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""",2,2,1,"""08:00""","""08:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.API N""","""UJEP""","""12.4.2024""","""""","""""","""R""",[2776],"""RNDr. Jan Krej…","""'RNDr. Jan Kre…","""Krejčí""",487024,"""""","""""","""RSKVARA""",""""""
479612,"""Programování -…","""KI""","""KPYT""","""""","""6259""",[6259],"""Jiří""","""Škvára""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""",6.13,30,5,2,"""Cvičení""","""Cv""","""A""","""Sobota""","""So""","""""",7,9,3,"""13:00""","""15:50""",15,15,"""Jiný""","""J""","""31867""","""N""","""N""","""Automaticky vy…","""2.PMVTBcKS""","""UJEP""","""13.4.2024""","""""","""""","""R""",[6259],"""RNDr. Jiří Škv…","""'RNDr. Jiří Šk…","""Škvára""",479612,"""""","""""","""RSKVARA""",""""""
491279,"""Datová a infor…","""KI""","""KDIB""","""""","""2776""",[2776],"""Jan""","""Krejčí""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""",7.07,12,7,4,"""Cvičení""","""Cv""","""A""","""Sobota""","""So""","""""",10,11,2,"""16:00""","""17:50""",15,15,"""Jiný""","""J""","""31988""","""N""","""N""","""Automaticky vy…","""1.KAPI N""","""UJEP""","""13.4.2024""","""""","""""","""R""",[2776],"""RNDr. Jan Krej…","""'RNDr. Jan Kre…","""Krejčí""",491279,"""""","""""","""RSKVARA""",""""""
487005,"""Programování p…","""KI""","""PRI""","""""","""2239""",[2239],"""Viktor""","""Maškov""","""doc. RNDr. Mgr…","""DrSc.""","""A""","""A""","""100""",2023,"""CP""",6.14,31,44,42,"""Přednáška""","""Př""","""A""","""Středa""","""St""","""""",2,3,2,"""08:00""","""09:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""2.API Bc,3.API…","""UJEP""","""17.4.2024""","""""","""""","""R""",[2239],"""doc. RNDr. Mgr…","""'doc. RNDr. Mg…","""Maškov""",487005,"""""","""""","""RSKVARA""",""""""


In [23]:
maly_rozvrh = rozvrh_by_kat.select(["zkratka", "vsichniUciteleUcitIdno", "typAkceZkr", "rok", "datum", "hodinaSkutOd", "hodinaSkutDo"])
maly_rozvrh = maly_rozvrh.with_columns(pl.col("vsichniUciteleUcitIdno")).explode("vsichniUciteleUcitIdno").rename({"vsichniUciteleUcitIdno": "idno"})
maly_rozvrh.head()

zkratka,idno,typAkceZkr,rok,datum,hodinaSkutOd,hodinaSkutDo
str,i64,str,i64,str,str,str
"""OPS""",650,"""Př""",2023,"""11.4.2024""","""14:00""","""14:50"""
"""DIB""",2776,"""Př""",2023,"""12.4.2024""","""08:00""","""08:50"""
"""KPYT""",6259,"""Cv""",2023,"""13.4.2024""","""13:00""","""15:50"""
"""KDIB""",2776,"""Cv""",2023,"""13.4.2024""","""16:00""","""17:50"""
"""PRI""",2239,"""Př""",2023,"""17.4.2024""","""08:00""","""09:50"""


In [24]:
# predmety_by_kat

In [25]:
from pprint import pprint
# pprint(predmety_by_kat.columns,compact=True)
predmety_by_kat.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""AFJ""",2023,"""Teorie automat…","""Teorie automat…","""A""","""N""",5,"""NE""","""""","""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…","[449, 2220]","""""","""""",null,"""""","""""","""""","""""","""""","""KI/AFJ1, KI/TA…","""""","""'Chytil M. Aut…","""""","""""","""""","""N""",2,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""V tomto předmě…","""Zkouška""","""ANO""","""Kombinovaná""","""zápočet písemn…","""1. Konečné aut…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""","""0"""
"""KI""","""AIT""",2023,"""Architektura a…","""Architektura a…","""A""","""N""",3,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""""","""""",null,"""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""","""""","""""","""KI/ZPP, KI/ZPS…","""""","""KI/IVP""","""'Velte, A. T.,…","""""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",2,"""HOD/TYD""","""Kurz je zaměře…","""Zkouška""","""NE""","""Kombinovaná""","""Zápočet: labor…","""1. IT a její ž…","""Základny počít…","""""","""""","""""","""Čeština""","""Tento kurz vzn…","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""ANE""",2023,"""Analogová elek…","""Analogová elek…","""A""","""N""",2,"""NE""","""""","""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""""","""""",null,"""""","""""","""""","""""","""KI/ZEL""","""""","""""","""'Tichý, M. Ele…","""""","""""","""""","""N""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Kurz je zaměře…","""Zápočet""","""NE""","""Kombinovaná""","""Ústní obhajoba…","""1. Pasivní ele…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""APC""",2023,"""Architektura p…","""Architektura p…","""A""","""N""",3,"""NE""","""""","""'doc. RNDr. Mg…","""'doc. RNDr. Mg…",[2239],"""'Ing. Pavel Ku…","""'Ing. Pavel Ku…","[2317, 2239]","""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""KI/DHW, KI/PPT…","""'Horák J. Hard…","""""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Tento předmět …","""Zkouška""","""NE""","""Kombinovaná""","""Zkouška je kom…","""1) Vývoj PC, g…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""",""

In [26]:
ciselnik_ucitelu = pl.read_csv("source_tables/ciselnik_ucitelu.csv").select("nazev", "key").rename({"nazev":"jmena", "key":"idno"})
ciselnik_ucitelu.head()

jmena,idno
str,i64
"""Abrahamová Pet…",8409
"""Ač Alexander, …",9316
"""Adamec Slavomí…",5432
"""Adámek Pavel, …",7443
"""Adam Julie, M.…",9249


In [27]:
male_predmety = predmety_by_kat.select(pl.col(["zkratka", "katedra", "rok", "nazev", "garantiUcitIdno", "prednasejici", "prednasejiciUcitIdno","cvicici", "cviciciUcitIdno","seminarici", "seminariciUcitIdno"]).where(pl.col("zkratka").is_unique()))
#male_predmety = male_predmety.with_columns(pl.col(["prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "garantiUcitIdno"]).str.split(", "))
male_predmety.filter(pl.col("zkratka") == "EPCA").head()

C:\Users\Matej\AppData\Local\Temp\ipykernel_15868\2091797618.py:1: DeprecationWarning: `where` is deprecated. Use `filter` instead.
  male_predmety = predmety_by_kat.select(pl.col(["zkratka", "katedra", "rok", "nazev", "garantiUcitIdno", "prednasejici", "prednasejiciUcitIdno","cvicici", "cviciciUcitIdno","seminarici", "seminariciUcitIdno"]).where(pl.col("zkratka").is_unique()))


zkratka,katedra,rok,nazev,garantiUcitIdno,prednasejici,prednasejiciUcitIdno,cvicici,cviciciUcitIdno,seminarici,seminariciUcitIdno
str,str,i64,str,list[i64],str,list[i64],str,list[i64],str,list[i64]


In [28]:
# přednášející nemá přednášku:
filtrovani_prednasejici = male_predmety.select("nazev", "zkratka", "prednasejiciUcitIdno").explode("prednasejiciUcitIdno")
prednasejici_jmena = male_predmety.select("prednasejici").rename({"prednasejici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
prednasejici_jmena = prednasejici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_prednasejici = filtrovani_prednasejici.with_columns(prednasejici_jmena).filter(
    pl.col("prednasejiciUcitIdno"
).is_not_null()).with_columns(
    pl.col("prednasejiciUcitIdno").alias("idno")
)
joined_prednasejici = filtrovani_prednasejici.join(maly_rozvrh, "idno", "left")
prednasejici_bez_prednasek = joined_prednasejici.filter(pl.col("typAkceZkr").is_null())
prednasejici_bez_prednasek.select("nazev", "zkratka", "prednasejiciUcitIdno", "jmena").sort("prednasejiciUcitIdno").write_excel("results/prednasejici_bez_prednasek.xlsx")
prednasejici_bez_prednasek.select("nazev", "zkratka", "prednasejiciUcitIdno", "jmena").sort("prednasejiciUcitIdno").write_csv("results_testing/prednasejici_bez_prednasek.csv")

In [29]:
male_prednasky = maly_rozvrh.filter(pl.col("typAkceZkr") == "Př")
joined_prednasky = male_prednasky.join(male_predmety.select("zkratka", "katedra", "nazev", "prednasejici", "prednasejiciUcitIdno"), "zkratka", "left").unique()
prednasky_bez_prednasejicich = joined_prednasky.filter(pl.col("idno").is_in(pl.col("prednasejiciUcitIdno")).not_())

prednasky_bez_prednasejicich = prednasky_bez_prednasejicich.join(ciselnik_ucitelu, "idno", "left").with_columns(pl.col("prednasejiciUcitIdno").cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

prednasky_bez_prednasejicich.write_csv("results_testing/prednasky_bez_prednasejicich.csv")
prednasky_bez_prednasejicich.write_excel("results/prednasky_bez_prednasejicich.xlsx")


print(prednasky_bez_prednasejicich.__len__())
prednasky_bez_prednasejicich.head(15)

51


zkratka,idno,typAkceZkr,rok,datum,hodinaSkutOd,hodinaSkutDo,katedra,nazev,prednasejici,prednasejiciUcitIdno,jmena
str,i64,str,i64,str,str,str,str,str,str,str,str
"""EAPR2""",null,"""Př""",2023,"""""","""00:00""","""00:00""",null,null,null,null,null
"""EDAV""",3416,"""Př""",2023,"""15.5.2024""","""15:00""","""15:50""",null,null,null,null,"""Babichev Sergi…"
"""EDAV""",6259,"""Př""",2023,"""15.5.2024""","""15:00""","""15:50""",null,null,null,null,"""Škvára Jiří, R…"
"""EDAV""",6973,"""Př""",2023,"""15.5.2024""","""15:00""","""15:50""",null,null,null,null,"""Beránek Pavel,…"
"""EDAV""",2220,"""Př""",2023,"""17.4.2024""","""15:00""","""15:50""",null,null,null,null,"""Škvor Jiří, RN…"
…,…,…,…,…,…,…,…,…,…,…,…
"""EDAV""",6259,"""Př""",2023,"""24.4.2024""","""15:00""","""15:50""",null,null,null,null,"""Škvára Jiří, R…"
"""EDAV""",2220,"""Př""",2023,"""15.5.2024""","""15:00""","""15:50""",null,null,null,null,"""Škvor Jiří, RN…"
"""EDAV""",6973,"""Př""",2023,"""24.4.2024""","""15:00""","""15:50""",null,null,null,null,"""Beránek Pavel,…"


In [30]:
# cvičící nemá cvičení:
filtrovani_cvicici = male_predmety.select("nazev", "zkratka", "cviciciUcitIdno").explode("cviciciUcitIdno")
cvicici_jmena = male_predmety.select("cvicici").rename({"cvicici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
cvicici_jmena = cvicici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_cvicici = filtrovani_cvicici.with_columns(cvicici_jmena).filter(
    pl.col("cviciciUcitIdno"
).is_not_null()).with_columns(
    pl.col("cviciciUcitIdno").alias("idno")
)
joined_cvicici = filtrovani_cvicici.join(maly_rozvrh, "idno", "left")
cvicici_bez_cviceni = joined_cvicici.filter(pl.col("typAkceZkr").is_null())
cvicici_bez_cviceni.select("nazev", "zkratka", "cviciciUcitIdno").sort("cviciciUcitIdno").write_excel("results/cvicici_bez_cviceni.xlsx")
cvicici_bez_cviceni.select("nazev", "zkratka", "cviciciUcitIdno", "jmena").sort("cviciciUcitIdno").write_csv("results_testing/cvicici_bez_cviceni.csv")

In [31]:
male_cviceni = maly_rozvrh.filter(pl.col("typAkceZkr") == "Cv")
joined_cviceni = male_cviceni.join(male_predmety.select("zkratka", "katedra", "nazev", "cvicici", "cviciciUcitIdno"), "zkratka", "left").unique()
cviceni_bez_cvicich = joined_cviceni.filter(pl.col("idno").is_in(pl.col("cviciciUcitIdno")).not_())

cviceni_bez_cvicich = cviceni_bez_cvicich.join(ciselnik_ucitelu, "idno", "left").with_columns(pl.col("cviciciUcitIdno").cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

cviceni_bez_cvicich.write_csv("results_testing/cviceni_bez_cvicich.csv")
cviceni_bez_cvicich.write_excel("results/cviceni_bez_cvicich.xlsx")

print(cviceni_bez_cvicich.__len__())
cviceni_bez_cvicich.head(15)

71


zkratka,idno,typAkceZkr,rok,datum,hodinaSkutOd,hodinaSkutDo,katedra,nazev,cvicici,cviciciUcitIdno,jmena
str,i64,str,i64,str,str,str,str,str,str,str,str
"""0033""",449,"""Cv""",2023,"""24.4.2024""","""14:00""","""15:50""",null,null,null,null,"""Barilla Jiří, …"
"""0033""",449,"""Cv""",2023,"""15.5.2024""","""14:00""","""15:50""",null,null,null,null,"""Barilla Jiří, …"
"""0033""",449,"""Cv""",2023,"""17.4.2024""","""14:00""","""15:50""",null,null,null,null,"""Barilla Jiří, …"
"""0121""",2776,"""Cv""",2023,"""""","""00:00""","""00:00""",null,null,null,null,"""Krejčí Jan, RN…"
"""0126""",2776,"""Cv""",2023,"""""","""00:00""","""00:00""",null,null,null,null,"""Krejčí Jan, RN…"
…,…,…,…,…,…,…,…,…,…,…,…
"""BOP""",2220,"""Cv""",2023,"""""","""00:00""","""00:00""","""KI""","""Odborná praxe""","""""",null,"""Škvor Jiří, RN…"
"""BP2O""",null,"""Cv""",2023,"""""","""00:00""","""00:00""",null,null,null,null,null
"""BPP""",null,"""Cv""",2023,"""""","""00:00""","""00:00""",null,null,null,null,null


In [32]:
# seminařicí nemá seminář:
filtrovani_seminarici = male_predmety.select("nazev", "zkratka", "seminariciUcitIdno").explode("seminariciUcitIdno")
seminarici_jmena = male_predmety.select("seminarici").rename({"seminarici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
seminarici_jmena = seminarici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_seminarici = filtrovani_seminarici.with_columns(seminarici_jmena).filter(
    pl.col("seminariciUcitIdno"
).is_not_null()).with_columns(
    pl.col("seminariciUcitIdno").alias("idno")
)
joined_seminarici = filtrovani_seminarici.join(maly_rozvrh, "idno", "left")
seminarici_bez_seminare = joined_seminarici.filter(pl.col("typAkceZkr").is_null())
seminarici_bez_seminare.select("nazev", "zkratka", "seminariciUcitIdno").sort("seminariciUcitIdno").write_excel("results/seminarici_bez_seminare.xlsx")

In [33]:
male_seminare = maly_rozvrh.filter(pl.col("typAkceZkr") == "Se")
joined_seminare = male_seminare.join(male_predmety.select("zkratka", "katedra", "nazev", "seminarici", "seminariciUcitIdno"), "zkratka", "left").unique()
seminare_bez_seminaricich = joined_seminare.filter(pl.col("idno").is_in(pl.col("seminariciUcitIdno")).not_()).unique()

seminare_bez_seminaricich = seminare_bez_seminaricich.join(ciselnik_ucitelu, "idno", "left").with_columns(pl.col("seminariciUcitIdno").cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

seminare_bez_seminaricich.write_csv("results_testing/seminare_bez_seminaricich.csv")
seminare_bez_seminaricich.write_excel("results/seminare_bez_seminaricich.xlsx")
print(seminare_bez_seminaricich.__len__())
seminare_bez_seminaricich.head(6)

117


zkratka,idno,typAkceZkr,rok,datum,hodinaSkutOd,hodinaSkutDo,katedra,nazev,seminarici,seminariciUcitIdno,jmena
str,i64,str,i64,str,str,str,str,str,str,str,str
"""0201""",2317,"""Se""",2023,"""17.4.2024""","""12:00""","""13:50""",null,null,null,null,"""Kuba Pavel, In…"
"""0201""",2317,"""Se""",2023,"""24.4.2024""","""12:00""","""13:50""",null,null,null,null,"""Kuba Pavel, In…"
"""0201""",2317,"""Se""",2023,"""15.5.2024""","""12:00""","""13:50""",null,null,null,null,"""Kuba Pavel, In…"
"""0204""",null,"""Se""",2023,"""30.4.2024""","""15:00""","""15:50""",null,null,null,null,null
"""0204""",null,"""Se""",2023,"""23.4.2024""","""15:00""","""16:50""",null,null,null,null,null
"""0204""",null,"""Se""",2023,"""14.5.2024""","""15:00""","""15:50""",null,null,null,null,null


In [34]:
predmety_by_kat.filter(pl.col("zkratka") == "KTZI2").head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""KTZI2""",2023,"""Teoretické zák…","""Teoretické zák…","""N""","""A""",3,"""NE""","""""","""'RNDr. Martin …","""'RNDr. Martin …",[264],"""'RNDr. Martin …","""'RNDr. Martin …","[264, 267]","""'RNDr. Martin …","""'RNDr. Martin …","[264, 267]","""""","""""",null,"""""","""""","""""","""""","""KI/KTZI1""","""""","""""","""'Demel, J. Gra…","""""","""""","""""","""N""",6,"""HOD/SEM""",4,"""HOD/SEM""",2,"""HOD/SEM""","""Kurz je koncip…","""Zápočet""","""NE""","""Kombinovaná""","""Požadavky k zá…","""1. Rekurence a…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""


### warning, když je mezi vyučujícími někdo bez odpovídajícího typu rozvrhové akce

Pro každého učitele vypíšu předměty, u kterých má mít nějakou akci.

Takže budu mít dataframe, kde bude učitel, zkratka, a jeden sloupeček za každý typ akce, kde učitel bude muset mít aspoň 1 narozvrhovaný typ akce s daným předmětem, kde je hodnota `True`


In [35]:
# vypíšu typy akcí
list(maly_rozvrh.select(pl.col("typAkceZkr").unique()))

[shape: (4,)
 Series: 'typAkceZkr' [str]
 [
 	"Zápočet"
 	"Se"
 	"Cv"
 	"Př"
 ]]

### garant předmětu by měl být někde mezi vyučujícími rozvrhových akcí a pokud má předmět přednášku, měl by být i mezi přednášejícími v rámci rozvrhových akcí

### Změny proti akreditacím.

### Porovnat revize předmětů

tj. parametry předmětů pro aktuální akademický rok a pro příští.

> jakou to má mít podobu?